## AuxTel test LTS-337-015 (Small Slew and Settle)

In this notebook, we check the speed and accuracy of the slew and settle of the telescope with a
change of 10 degrees.  Desired specs:

| Description | Value       | Unit          |   Name     |
| :---        |    :----:   |       :----:  |       ---: |
|The pointing accuracy requirement for small slews is:    | 2.5       | Arcsec RMS   |Aux_Tel_Sm_Slew_Abs_Pointing|
The small elevation slew requirement for the Auxiliary Telescope used to define the Aux_Tel_Sm_Slew_Abs_Pointing requirement| 10| Degrees |Aux_Tel_Sm_Elev_Slew|
|The small azimuth slew requirement for the Auxiliary Telescope used to define the Aux_Tel_Sm_Slew_Abs_Pointing requirement.  |10       | Degrees     |Aux_Tel_Sm_Az_Slew|
|The allotted time to perform a slew and settle by Aux_Tel_Sm_Elev_Slew and Aux_Tel_Sm_Az_Slew by the Auxiliary Telescope and meet the Aux_Tel_Sm_Slew_Abs_Pointing pointing requirement.      | 10|Seconds|Aux_Tel_Smslew_AcqTime|

In [1]:
import sys, time, os, asyncio

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
from lsst_efd_client import EfdClient


Bad key "text.kerning_factor" on line 4 in
/opt/lsst/software/stack/conda/miniconda3-py37_4.8.2/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Set Cerro Pachon location
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [3]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [4]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [5]:
# Get EFD client and bring in Lupton's unpacking code
client = EfdClient('summit_efd')

def merge_packed_time_series(packed_dataframe, base_field, stride=1, 
                             ref_timestamp_col="cRIO_timestamp", internal_time_scale="tai"):
    """Select fields that are time samples and unpack them into a dataframe.
            Parameters
            ----------
            packedDF : `pandas.DataFrame`
                packed data frame containing the desired data
            base_field :  `str`
                Base field name that will be expanded to query all
                vector entries.
            stride : `int`, optional
                Only use every stride value when unpacking.  Must be a factor
                of the number of packed values.
                (1 by default)
            ref_timestamp_col : `str`, optional
                Name of the field name to use to assign timestamps to unpacked
                vector fields (default is 'cRIO_timestamp').
            internal_time_scale : `str`, optional
                Time scale to use when converting times to internal formats
                ('tai' by default). Equivalent to EfdClient.internal_scale
        Returns
            -------
            result : `pandas.DataFrame`
                A `pandas.DataFrame` containing the results of the query.
            """
    
    packed_fields = [k for k in packed_dataframe.keys() if k.startswith(base_field)]
    packed_fields = sorted(packed_fields, key=lambda k: int(k[len(base_field):]))  # sort by pack ID
    npack = len(packed_fields)
    if npack%stride != 0:
        raise RuntimeError(f"Stride must be a factor of the number of packed fields: {stride} v. {npack}")
    packed_len = len(packed_dataframe)
    n_used = npack//stride   # number of raw fields being used
    output = np.empty(n_used*packed_len)
    times = np.empty_like(output, dtype=packed_dataframe[ref_timestamp_col][0])
    
    if packed_len == 1:
        dt = 0
    else:
        dt = (packed_dataframe[ref_timestamp_col][1] - packed_dataframe[ref_timestamp_col][0])/npack
    for i in range(0, npack, stride):
        i0 = i//stride
        output[i0::n_used] = packed_dataframe[f"{base_field}{i}"]
        times[i0::n_used] = packed_dataframe[ref_timestamp_col] + i*dt
     
    timestamps = Time(times, format='unix', scale=internal_time_scale).datetime64
    return pd.DataFrame({base_field:output, "times":times}, index=timestamps)

Starting new HTTPS connection (1): roundtable.lsst.codes:443
https://roundtable.lsst.codes:443 "GET /segwarides/ HTTP/1.1" 200 253
Starting new HTTPS connection (1): roundtable.lsst.codes:443
https://roundtable.lsst.codes:443 "GET /segwarides/creds/summit_efd HTTP/1.1" 200 92


In [6]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.09 sec
Read 8 history items for RemoteEvent(ATHexapod, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATHexapod, 0, authList)
Read 100 history items for RemoteEvent(ATHexapod, 0, detailedState)
Read 1 history items for RemoteEvent(ATHexapod, 0, errorCode)
Read 100 history items for RemoteEvent(ATHexapod, 0, heartbeat)
Read 100 history items for RemoteEvent(ATHexapod, 0, inPosition)
Read 1 history items for RemoteEvent(ATHexapod, 0, logLevel)
Read 3 history items for RemoteEvent(ATHexapod, 0, logMessage)
Read 55 history items for RemoteEvent(ATHexapod, 0, positionUpdate)
Read 1 h

[[None, None, None, None, None, None, None], [None, None, None, None]]

In [7]:
# enable components if required
await atcs.enable()
await latiss.enable()

Enabling all components
Gathering settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for atmcs.
Complete settings for atptg.
Complete settings for ataos.
Complete settings for atpneumatics.
Complete settings for athexapod.
Complete settings for atdome.
Complete settings for atdometrajectory.
Settings versions: {'atmcs': '                                                                                                                               ', 'atptg': '', 'ataos': 'current', 'atpneumatics': '                                                                                                                               ', 'athexapod': 'summit', 'atdome': 'test', 'atdometrajectory': ''}
[atmcs]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atptg]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ataos]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atpneumatics]::[<State.STANDBY: 5>, <State.DISABLED: 

In [8]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [9]:
# turn on ATAOS corrections just to make sure the mirror is under air
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=False)

In [10]:
# Ensure we're using Nasmyth 2
await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [40]:
# point telescope to desired starting position
start_az=0
start_el=70
start_rot_pa=0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

Sending command
Stop tracking.
Unknown tracking state: 9.
Unknown tracking state: 10.
In Position: True.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = -000.000 deg; delta Az= +000.000 deg; delta N1 = -000.000 deg; delta N2 = -000.000 deg 
Axes in position.


In [71]:
#declare offset sizes for tests

# For small slew and settle
d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 0 # degrees

"""
# For large slew and settle
d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 90 # degrees

# What we actually care about for the survey! So measure this too
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees
"""

'\n# For large slew and settle\nd_az= 90 # degrees\nd_el = 60 # degrees\nd_rot= 90 # degrees\n\n# What we actually care about for the survey! So measure this too\nd_az= 3.5 # degrees\nd_el = 3.5 # degrees\nd_rot= 3.5 # degrees\n'

In [76]:
# get RA/DEC of current telescope Alt/Az position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await atcs.rem.atptg.tel_timeAndDate.next(flush=True, timeout=2)
# This should be TAI and not UTC... so will be 37s off system clock seconds ??
curr_time_atptg = Time(time_data.mjd, format="mjd")

coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))

orig az 0.0 deg and el 70.0 deg
Current Position is: 
 <AltAz Coordinate (obstime=59255.79257827408, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (0., 70.)>
Current Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (355.38776874, -10.35775162)>
target az 10.0 deg and el 80.0 deg
Target Position is: 
 <AltAz Coordinate (obstime=59255.79257827408, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (10., 80.)>
Target Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (357.23081801, -20.49672226)>


In [77]:
# This is where the tests are carried out.
times = []
ra_accuracies = []
dec_accuracies = []

# Slew to starting position
await atcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image to get some header data
await latiss.take_engtest(exptime=1)

# Now slew to new target
print('Starting to Slew to target')
start_time = Time(Time.now(), format='fits', scale='tai')
await atcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec),
                      rot=d_rot, slew_timeout=240., 
                      stop_before_slew=False, wait_settle=False)
end_time = Time(Time.now(), format='fits', scale='tai')
slew_time = TimeDelta(end_time - start_time, format='sec').value
print(f'Time to slew is {slew_time} seconds')
times.append(slew_time)
await latiss.take_engtest(exptime=1)

t_end = end_time + TimeDelta(2, format='sec')
nsec = 2
mount_position = await client.select_time_series("lsst.sal.ATPtg.mount_positions", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
ra = merge_packed_time_series(mount_position, 'ra', stride=1)
dec = merge_packed_time_series(mount_position, 'declination', stride=1)
ra_vals = np.array(ra.values.tolist())[:,0]
dec_vals = np.array(dec.values.tolist())[:,0]
# Errors in arcsec
ra_rms = np.sqrt(np.mean((ra_vals*15.0 - ra_dec_target.ra.value)**2)) * 3600.0 
dec_rms = np.sqrt(np.mean((dec_vals - ra_dec_target.dec.value)**2)) * 3600.0 
ra_accuracies.append(ra_rms)
dec_accuracies.append(dec_rms)
print(f"RA rms = {ra_rms}. Dec rms = {dec_rms}.")

# Repeat this going back to origin

print('Starting to Slew to target')
start_time = Time(Time.now(), format='fits', scale='tai')
await atcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)

end_time = Time(Time.now(), format='fits', scale='tai')
slew_time = TimeDelta(end_time - start_time, format='sec').value
print(f'Time to slew is {slew_time} seconds')
times.append(slew_time)
await latiss.take_engtest(exptime=1)

t_end = end_time + TimeDelta(2, format='sec')
nsec = 2
mount_position = await client.select_time_series("lsst.sal.ATPtg.mount_positions", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
ra = merge_packed_time_series(mount_position, 'ra', stride=1)
dec = merge_packed_time_series(mount_position, 'declination', stride=1)
ra_vals = np.array(ra.values.tolist())[:,0]
dec_vals = np.array(dec.values.tolist())[:,0]
# Errors in arcsec
ra_rms = np.sqrt(np.mean((ra_vals*15.0 - ra_dec.ra.value)**2)) * 3600.0 
dec_rms = np.sqrt(np.mean((dec_vals - ra_dec.dec.value)**2)) * 3600.0 
ra_accuracies.append(ra_rms)
dec_accuracies.append(dec_rms)
print(f"RA rms = {ra_rms}. Dec rms = {dec_rms}.")



Auto sky angle: 0.0 deg
Sending command
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
Got False
Telescope not in position
[Telescope] delta Alt = +000.000 deg; delta Az= +001.040 deg; delta N1 = +000.000 deg; delta N2 = -000.904 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= +000.359 deg; delta N1 = +000.000 deg; delta N2 = -000.054 deg 
Got True
Telescope in position.
track for 2s
Generating group_id
ENGTEST 0001 - 0001
Starting to Slew to target
Auto sky angle: 0.0 deg
Sending command
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
Got False
Telescope not in position
[Telescope] delta Alt = +009.979 deg; delta Az= +010.258 deg; delta N1 = -000.000 deg; delta N2 = +000.486 deg 
[Telescope] de

In [80]:
# Now check to see if the specs are met:
Aux_Tel_Smslew_AcqTime = 10.0
if max(times) < Aux_Tel_Smslew_AcqTime:
    print(f"Aux_Tel_Smslew_AcqTime passed.  Spec = {Aux_Tel_Smslew_AcqTime}.\
    Measured = {max(times)} ")
else:
    print(f"Aux_Tel_Smslew_AcqTime failed!  Spec = {Aux_Tel_Smslew_AcqTime}.\
    Measured = {max(times)} ")

Aux_Tel_Sm_Slew_Abs_Pointing = 2.5
worst_case = max(ra_accuracies + dec_accuracies)
if worst_case < Aux_Tel_Sm_Slew_Abs_Pointing:
    print(f"Aux_Tel_Sm_Slew_Abs_Pointing passed.  Spec = {Aux_Tel_Sm_Slew_Abs_Pointing}. \
    Measured = {worst_case} ")
else:
    print(f"Aux_Tel_Sm_Slew_Abs_Pointing failed!  Spec = {Aux_Tel_Sm_Slew_Abs_Pointing}. \
    Measured = {worst_case} ")


Aux_Tel_Smslew_AcqTime failed!  Spec = 10.0.    Measured = 10.529597000009261 
Aux_Tel_Sm_Slew_Abs_Pointing failed!  Spec = 2.5.     Measured = 577.5615675990892 


In [81]:
# For shutdown of system
await atcs.stop_tracking()

Stop tracking.
Unknown tracking state: 9.
Unknown tracking state: 10.
In Position: True.


In [82]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [83]:
# Putting everything back in standby.
await atcs.shutdown()

Disabling ATAOS corrections
Cover state <MirrorCoverState.CLOSED: 6>
M1 cover already closed.
M1 vent state <VentsPosition.CLOSED: 1>
M1 vents already closed.
Close dome.
ATDome Shutter Door is already closed. Ignoring.
Slew dome to Park position.


RuntimeError: ATDome is deactivated. Activate it by setting `check.atdome=True` before slewing.In some cases users deactivate a component on purpose.Make sure it is clear to operate the dome before doing so.

In [84]:
# Putting everything back in standby.
# Need to do it one at a time, since ATDome fails with check=False.
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.ataos, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [90]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atheaderservice, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atarchiver, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [92]:
await atcs.rem.atdome.cmd_start.set_start(settingsToApply="test", timeout=30)

In [93]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply="test")

[<State.DISABLED: 1>, <State.STANDBY: 5>]